In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
from os.path import join
from tqdm import tqdm
import pandas as pd
from os.path import expanduser
import sys
from typing import List
import numpy as np
import joblib
from pprint import pprint
import imodelsx.util
from os.path import dirname
import pickle as pkl
import json
from copy import deepcopy
from numpy.linalg import norm
from math import ceil
from imodelsx.qaemb.qaemb import QAEmb, get_sample_questions_and_examples
from neuro.ecog.config import STORIES_POPULAR, STORIES_UNPOPULAR, ECOG_DIR
import mne
import h5py
import torch
import base64

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
extract = __import__('00_extract')
from nilearn.plotting import plot_markers
from mne_bids import BIDSPath

from himalaya.backend import set_backend, get_backend
from himalaya.ridge import RidgeCV
from himalaya.scoring import correlation_score

from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from neuro.ecog.config import ECOG_DIR

In [ ]:
bids_root = join(ECOG_DIR, 'podcasts_data', 'ds005574/')

# Download the transcript, if required
# transcript_path = f"{bids_root}stimuli/gpt2-xl/transcript.tsv"
transcript_path = join(bids_root, "stimuli/syntactic/transcript.tsv")

# Load transcript
df_contextual = pd.read_csv(transcript_path, sep="\t", index_col=0)
df_contextual.head()

df_word = df_contextual.groupby("word_idx").agg(
    dict(word="first", start="first", end="last"))
df_word = df_word.rename(columns={'word': 'text'})

In [ ]:
out_dir = join(ECOG_DIR, 'data', 'transcripts')
os.makedirs(out_dir, exist_ok=True)
df_word.to_csv(join(out_dir, '___podcasts-story___', 'features.csv'))

## load wav file

In [ ]:
from pydub import AudioSegment

# Load the WAV file
audio = AudioSegment.from_wav("podcast.wav")

In [63]:
# word-level

# Split the audio into segments based on the word timings
word_buffer = 0.275
# include word_buffer seconds before and after, unless it overlaps with the previous or next word
backup_buffer = 0.01
# if it overlaps, use the end of the previous word or the start of the next word with a backup_buffer offset
df_word['end_prev_word'] = df_word['end'].shift(1, fill_value=0)
df_word['word_wav_start'] = df_word.apply(lambda row: max(
    row['start'] - word_buffer, row['end_prev_word'] + backup_buffer), axis=1)
df_word['start_next_word'] = df_word['start'].shift(-1, fill_value=10000000)
df_word['word_wav_end'] = df_word.apply(lambda row: min(
    row['end'] + word_buffer, row['start_next_word'] - backup_buffer), axis=1)

df_word['6sec_wav_start'] = (df_word['word_wav_end'] - 6).clip(lower=0)
df_word['3sec_wav_start'] = (df_word['word_wav_end'] - 3).clip(lower=0)
df_word['1.5sec_wav_start'] = (df_word['word_wav_end'] - 1.5).clip(lower=0)

In [ ]:
from collections import defaultdict


for k in ['segments_word', 'segments_3sec', 'segments_1.5sec', 'segments_6sec']:
    os.makedirs(k, exist_ok=True)

d = defaultdict(list)
for idx, row in list(tqdm(df_word.iterrows())):
    segment_word = audio[1000 * row['word_wav_start']: 1000 * row['word_wav_end']]
    segment_word.export(f"segments_word/segment_{idx}.wav", format="wav")

    segment_1_5sec = audio[1000 * row['1.5sec_wav_start']: 1000 * row['word_wav_end']]

    segment_1_5sec.export(f"segments_1.5sec/segment_{idx}.wav", format="wav")

    segment_3sec = audio[1000 * row['3sec_wav_start']: 1000 * row['word_wav_end']]
    segment_3sec.export(f"segments_3sec/segment_{idx}.wav", format="wav")

    segment_6sec = audio[1000 * row['6sec_wav_start']: 1000 * row['word_wav_end']]
    segment_6sec.export(f"segments_6sec/segment_{idx}.wav", format="wav")

0it [00:00, ?it/s]

5136it [00:01, 4701.09it/s] 
